# Foreign Born Population

## Setup

### Packages

In [14]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import xlsxwriter
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")


## Getting catalog from AGO

In [15]:
# Define the directory to save the data
data_path = Path('inputData/insightsCatalog')
data_path.mkdir(parents=True, exist_ok=True)

# Access the CSV item by ID
catalog_item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
csv_item = gis.content.get(catalog_item_id)

# Download the CSV file
catalog_path = csv_item.download(save_path=data_path)

print(f"CSV file downloaded successfully: {catalog_path}")

CSV file downloaded successfully: inputData\insightsCatalog\morpc_insights_catalog.xlsx


### Paths

In [16]:
INPUT_DATA_PATH = "inputData/TEMPinput.csv"

TEMP_OUTPUT_CSV_PATH ="outputData/TEMPoutput.csv"

REGIONAL_SUMMARY_DATA_PATH = "outputData/TEMPoutput_15Region.csv"

REGIONAL_SUMMARY_DATA_LABELED_PATH = "outputData/TEMPoutput_labeled_15Region.csv"

data1 = pd.read_csv(INPUT_DATA_PATH)

data1.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

data1.head()

,STATE,COUNTY,Estimate!!Total:,Estimate!!Total:!!Europe:,Estimate!!Total:!!Europe:!!Northern Europe:,Estimate!!Total:!!Europe:!!Northern Europe:!!Denmark,Estimate!!Total:!!Europe:!!Northern Europe:!!Ireland,Estimate!!Total:!!Europe:!!Northern Europe:!!Norway,Estimate!!Total:!!Europe:!!Northern Europe:!!Sweden,Estimate!!Total:!!Europe:!!Northern Europe:!!United Kingdom (inc. Crown Dependencies):,...,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Colombia,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Ecuador,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Guyana,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Peru,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Uruguay,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Venezuela,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Other South America,Estimate!!Total:!!Americas:!!Northern America:,Estimate!!Total:!!Americas:!!Northern America:!!Canada,Estimate!!Total:!!Americas:!!Northern America:!!Other Northern America
0,Ohio,Adams,287,50,50,0,0,0,0,50,...,0,0,0,0,0,0,0,15,15,0
1,Ohio,Allen,1856,203,89,9,0,0,1,79,...,2,0,0,14,0,0,0,73,73,0
2,Ohio,Ashland,665,149,39,0,0,0,0,39,...,0,0,0,18,0,0,0,16,16,0
3,Ohio,Ashtabula,1579,522,53,0,5,0,0,48,...,0,0,0,24,0,47,0,193,193,0
4,Ohio,Athens,2451,270,74,0,6,0,0,53,...,10,0,34,31,0,56,0,51,51,0


## Filtering

### Filtering for 15 county

In [17]:
# List of counties to filter by
selected_counties = [
    "Delaware", "Fairfield", "Fayette", "Franklin", "Hocking", "Knox", "Licking", 
    "Logan", "Madison", "Marion", "Morrow", "Perry", "Pickaway", "Ross", "Union"
]

# Filter the dataset for the selected counties
data2 = data1[data1['COUNTY'].isin(selected_counties)]

data2.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

# Display the first few rows of the filtered data
data2.head()

,STATE,COUNTY,Estimate!!Total:,Estimate!!Total:!!Europe:,Estimate!!Total:!!Europe:!!Northern Europe:,Estimate!!Total:!!Europe:!!Northern Europe:!!Denmark,Estimate!!Total:!!Europe:!!Northern Europe:!!Ireland,Estimate!!Total:!!Europe:!!Northern Europe:!!Norway,Estimate!!Total:!!Europe:!!Northern Europe:!!Sweden,Estimate!!Total:!!Europe:!!Northern Europe:!!United Kingdom (inc. Crown Dependencies):,...,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Colombia,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Ecuador,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Guyana,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Peru,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Uruguay,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Venezuela,Estimate!!Total:!!Americas:!!Latin America:!!South America:!!Other South America,Estimate!!Total:!!Americas:!!Northern America:,Estimate!!Total:!!Americas:!!Northern America:!!Canada,Estimate!!Total:!!Americas:!!Northern America:!!Other Northern America
20,Ohio,Delaware,18278,2025,656,0,97,0,68,491,...,144,34,0,82,0,87,0,604,604,0
22,Ohio,Fairfield,6479,733,129,15,0,0,23,91,...,69,0,2,35,0,0,0,147,147,0
23,Ohio,Fayette,620,78,8,0,0,0,0,8,...,0,0,0,0,0,0,0,0,0,0
24,Ohio,Franklin,158798,11607,2168,98,89,32,0,1920,...,906,700,187,363,25,1634,12,2168,2147,21
36,Ohio,Hocking,192,92,41,0,0,0,0,41,...,0,0,0,0,0,0,0,0,0,0


### Reshaping for long-form

In [18]:
# Reshape the data to a long format with `STATE` and `COUNTY` as identifying columns
long_data = data2.melt(id_vars=["STATE", "COUNTY"], 
                      var_name="VARIABLE", 
                      value_name="POP")
long_data.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

# Display the first few rows of the long data
long_data.head()

,STATE,COUNTY,VARIABLE,POP
0,Ohio,Delaware,Estimate!!Total:,18278
1,Ohio,Fairfield,Estimate!!Total:,6479
2,Ohio,Fayette,Estimate!!Total:,620
3,Ohio,Franklin,Estimate!!Total:,158798
4,Ohio,Hocking,Estimate!!Total:,192


### Summing by 15 County

In [19]:
# Group by the relevant columns and sum the 'POP' variable
data2_1 = long_data.groupby(
    ['VARIABLE']
)['POP'].sum().reset_index()

# Add a new column for the COUNTY identifier
data2_1['COUNTY'] = '15County'

# Rearrange columns to match the original dataset order
data2_1 = data2_1[['COUNTY', 'VARIABLE', 'POP']]

# Append the new '15County' entries to the filtered dataset
data3 = pd.concat([long_data, data2_1], ignore_index=True)

data3.to_csv(REGIONAL_SUMMARY_DATA_PATH, index=False)

# Display the first few rows of the updated data
data3.head()

,STATE,COUNTY,VARIABLE,POP
0,Ohio,Delaware,Estimate!!Total:,18278
1,Ohio,Fairfield,Estimate!!Total:,6479
2,Ohio,Fayette,Estimate!!Total:,620
3,Ohio,Franklin,Estimate!!Total:,158798
4,Ohio,Hocking,Estimate!!Total:,192


### Combining for aggregated geographies

In [20]:
# List of counties to filter by
selected_vars = [
    "Estimate!!Total:!!Asia:!!South Central Asia:", "Estimate!!Total:!!Americas:!!Latin America:", "Estimate!!Total:!!Africa:!!Eastern Africa:", "Estimate!!Total:!!Europe:!!Northern Europe:", "Estimate!!Total:!!Europe:!!Western Europe:", "Estimate!!Total:!!Europe:!!Southern Europe:", "Estimate!!Total:!!Europe:!!Eastern Europe:", 
    "Estimate!!Total:!!Asia:!!Eastern Asia:", "Estimate!!Total:!!Asia:!!Western Asia:", "Estimate!!Total:!!Africa:!!Middle Africa:", "Estimate!!Total:!!Africa:!!Northern Africa:", "Estimate!!Total:!!Africa:!!Southern Africa:", "Estimate!!Total:!!Africa:!!Western Africa:", "Estimate!!Total:!!Americas:!!Latin America:",
    "Estimate!!Total:!!Americas:!!Northern America:", "Estimate!!Total:!!Oceania:!!Australia and New Zealand Subregion:", "Estimate!!Total:!!Asia:!!South Eastern Asia:", "Estimate!!Total:!!Europe:!!Europe, n.e.c.", "Estimate!!Total:!!Oceania:!!Fiji", "Estimate!!Total:!!Asia:!!Asia, n.e.c.", "Estimate!!Total:!!Africa:!!Africa, n.e.c.",
    "Estimate!!Total:!!Oceania:!!Oceania, n.e.c."
]


# Filter the dataset for the selected counties
data4 = data3[data3['VARIABLE'].isin(selected_vars)]

data4.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

# Display the first few rows of the filtered data
data4.head()

,STATE,COUNTY,VARIABLE,POP
30,Ohio,Delaware,Estimate!!Total:!!Europe:!!Northern Europe:,656
31,Ohio,Fairfield,Estimate!!Total:!!Europe:!!Northern Europe:,129
32,Ohio,Fayette,Estimate!!Total:!!Europe:!!Northern Europe:,8
33,Ohio,Franklin,Estimate!!Total:!!Europe:!!Northern Europe:,2168
34,Ohio,Hocking,Estimate!!Total:!!Europe:!!Northern Europe:,41


In [21]:
# Define a comprehensive mapping of race abbreviations to full race names
race_mapping = {
    "Estimate!!Total:!!Asia:!!South Central Asia:": "South Central Asia",
    "Estimate!!Total:!!Americas:!!Latin America:": "Latin America",
    "Estimate!!Total:!!Africa:!!Eastern Africa:": "Eastern Africa",
    "Estimate!!Total:!!Asia:!!Eastern Asia:": "Eastern Asia",
    "Estimate!!Total:!!Africa:!!Western Africa:": "Western Africa",
    "Estimate!!Total:!!Asia:!!South Eastern Asia:": "South Eastern Asia",
    "Estimate!!Total:!!Europe:!!Eastern Europe:": "Eastern Europe",
    "Estimate!!Total:!!Asia:!!Western Asia:": "Western Asia",
    "Estimate!!Total:!!Africa:!!Northern Africa:": "Northern Africa",
    "Estimate!!Total:!!Europe:!!Northern Europe:": "Northern Europe",
    "Estimate!!Total:!!Europe:!!Western Europe:": "Western Europe",
    "Estimate!!Total:!!Americas:!!Northern America:": "Northern America",
    "Estimate!!Total:!!Africa:!!Middle Africa:": "Middle Africa",
    "Estimate!!Total:!!Africa:!!Africa, n.e.c.": "Africa, other",
    "Estimate!!Total:!!Europe:!!Southern Europe:": "Southern Europe",
    "Estimate!!Total:!!Africa:!!Southern Africa:": "Southern Africa",
    "Estimate!!Total:!!Oceania:!!Australia and New Zealand Subregion:": "Australia and New Zealand",
    "Estimate!!Total:!!Asia:!!Asia, n.e.c.": "Asia, other",
    "Estimate!!Total:!!Oceania:!!Oceania, n.e.c.": "Oceania, other",
    "Estimate!!Total:!!Oceania:!!Fiji": "Fiji",
    "Estimate!!Total:!!Europe:!!Europe, n.e.c.": "Europe, other"
}

# Apply the mapping to the 'RACE' column
data4['VARIABLE'] = data4['VARIABLE'].map(race_mapping).fillna(data4['VARIABLE'])

data4.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

C:\Users\ogwynn\AppData\Local\Temp\ipykernel_24200\3139221840.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['VARIABLE'] = data4['VARIABLE'].map(race_mapping).fillna(data4['VARIABLE'])


## Garphing

### Creating images for each region

In [22]:
# Set up output directory
output_dir = 'outputData/plots'

os.makedirs(output_dir, exist_ok=True)

# Load the data from the CSV file
data = pd.read_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH)

data['POP']= data['POP']/1000


# Get unique counties
unique_counties = data['COUNTY'].unique()

# Create and save bar charts for each county
for county in unique_counties:
    county_data = data[data['COUNTY'] == county].sort_values(by='POP', ascending=False)
    plt.figure(figsize=(10, 6))
    plt.barh(county_data['VARIABLE'], county_data['POP'], color='green')
    plt.xlabel('Foreign-born Population (thousands)', fontsize=22)
    #plt.title(f'Foreign-born Population in {county} County', fontsize=20)
    plt.yticks(fontsize=18)  # Adjust fontsize to your preference
    plt.xticks(fontsize=18)  # Adjust fontsize to your preference

    plt.gca().invert_yaxis()  # Invert y-axis to match example chart order
    plt.tight_layout()

    # Save the plot as a PNG file and add to mapping
    plot_path = os.path.join(output_dir, f"{county}_foreign_born_population.png")
    
    # Save the figure for review
    plt.savefig(plot_path)
    plt.close()



### Creating Excel sheet and viz for each region

In [23]:
# Load the uploaded file to review its content
file_path =REGIONAL_SUMMARY_DATA_LABELED_PATH

# Load the data from the CSV file
data = pd.read_csv(file_path)

# Create an Excel writer object
with pd.ExcelWriter('outputData/plots/county_line_charts.xlsx', engine='xlsxwriter') as writer:
    
    # Loop through each unique county and create a chart
    for county in data['COUNTY'].unique():
        county_data = data[data['COUNTY'] == county]
        
        # Ensure 'YEAR', 'POP', and 'RACE' columns are present
        county_data = county_data[['VARIABLE', 'POP']]
        
        # Aggregate to handle duplicates by taking the mean or sum
        county_data = county_data.groupby(['VARIABLE'], as_index=False).agg({'POP': 'mean'})
        
        # Pivot data to have 'RACE' as columns for plotting
        county_data_pivot = county_data.pivot(index='POP', columns='VARIABLE', values='POP')
        
        # Create a line chart for the current county with a line for each RACE
        morpc.data_chart_to_excel(
            df=county_data_pivot,
            writer=writer,
            sheet_name=county,
            chartType='line',
            chartOptions={
                "titles": {
                    "chartTitle": f"{county} Foreign-born Population",
                    "xTitle": "Foreign-born Population (thousands)"
                },
                "legendOptions": {
                    "position": "bottom"
                }
            }
        )


### Uploading images to AGO

In [24]:
import time
from arcgis.gis import GIS
import os
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Directory where images are saved
output_dir = 'outputData/plots'

# Initialize dictionary to store county-item mappings
county_item_mapping = {}

# Loop through each file in the directory and upload it
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "tags": "county, population, race, demographics",
            "description": f"Foreign born population for {county_name} county.",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item


        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Store item ID in the dictionary
            county_item_mapping[county_name] = f"https://morpc.maps.arcgis.com/sharing/rest/content/items/{image_item.id}/data"
            
            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_df['TilesetID'] = 15
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)



Deleted existing item with title: 15County foreign born population


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15County_foreign_born_population.png with item ID: 05d1783be7c14b3ab4b7e0324795509d and set it to public
Deleted existing item with title: Delaware foreign born population


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Delaware_foreign_born_population.png with item ID: dd0df6ab55fc4bbc9e861324308129e0 and set it to public
Deleted existing item with title: Fairfield foreign born population


...

Uploaded Pickaway_foreign_born_population.png with item ID: 62b1612d6d3d4e5eac8c44acaee384fd and set it to public
Deleted existing item with title: Ross foreign born population


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Ross_foreign_born_population.png with item ID: d28ca19928044de99e6cbe77e58a3a3b and set it to public
Deleted existing item with title: Union foreign born population


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_foreign_born_population.png with item ID: 04cc3ec025514ceaa66790583429589a and set it to public


### Update Metadata Local

In [25]:
import pandas as pd

# Load the catalog data
catalog_df = pd.read_excel(catalog_path)

# Load the mapping data
mapping_df = pd.read_csv(mapping_file_path)

mapping_df['COUNTY'] = mapping_df['COUNTY'].replace('15-County', '15County')

# Merge the datasets based on the key pairs ('COUNTY', 'TilesetID') in mapping_df 
# and ('GeographyName', 'TilesetID') in catalog_df
merged_df = catalog_df.merge(
    mapping_df,
    left_on=['GeographyName', 'TilesetID'],
    right_on=['COUNTY', 'TilesetID'],
    how='left'
)

# Update 'ThumbnailURL' with 'itemUitemIDRL' where there is a match
merged_df['ThumbnailURL'] = merged_df['itemID'].combine_first(merged_df['ThumbnailURL'])

# Drop unnecessary columns from the merged dataset
merged_df = merged_df.drop(columns=['COUNTY', 'itemID'])

# Save the updated catalog back to an Excel file
updated_catalog_path = "outputData/morpc_insights_catalog_updated.xlsx"
merged_df.to_excel(updated_catalog_path, index=False)

# Provide the file to the user
print(f"Updated catalog saved at: {updated_catalog_path}")


Updated catalog saved at: outputData/morpc_insights_catalog_updated.xlsx


In [26]:
# Define the item ID of the Excel file in AGO
ITEM_ID = "c396b9d0db724d1f870fa47f1d1dc1ce"

# Path to the updated Excel file
updated_catalog_path = "outputData/morpc_insights_catalog_updated.xlsx"

# Get the existing item
item = gis.content.get(ITEM_ID)

if item:
    print(f"Found item: {item.title}")

    # Update the item with the new file
    item.update(
        item_properties={"title": item.title},  # Keeps the existing metadata
        data=updated_catalog_path  # Replaces the file
    )
    
    print("Successfully updated the Excel file in AGO.")
else:
    print("Item not found. Check the item ID and your permissions.")


Found item: morpc_insights_catalog
Successfully updated the Excel file in AGO.
